In [ ]:
# # stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# isort: off
# stdlib
from os import environ as env
import os
import sys

# syft absolute
import syft as sy

# from syft import get_helpers  # noqa: F401
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()

# third party
from email_helpers import get_email_server
# isort: on

# Launch & login

In [ ]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server(reset=True)

In [ ]:
root_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

# Create new admin client

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [ ]:
# create
root_client.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

# We cannot delete the root client

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

In [ ]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [ ]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

In [ ]:
root_client.users.delete(new_user_id2)

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()